# PROBLEM 2 : KMeans on data
Using Euclidian distance or dot product similarity (choose one per dataset, you can try other similarity metrics). <br>
A) run KMeans on the MNIST Dataset, try K=10 <br>
B) run KMeans on the FASHION Dataset, try K=10 <br>
C) run KMeans on the 20NG Dataset, try K=20 <br>
You can use a library for distance/similarity but you have to implement your own kmeans (EM steps, termination criteria etc). <br>
For all three datasets, evaluate the KMeans objective for a higher K (for example double) or smaller K(for example half). <br>
For all three datasets, evaluate external clustering performance using data labels and performance metrics Purity and Gini Index (see [A] book section 6.9.2).

In [ ]:
# Kmeans.py
import numpy as np

class KMeans:
    def __init__(self,k,dist_type,iters,num_of_true_lbls):
        self.k = k # number of clusters
        self.dist_type = dist_type
        self.iters = iters
        self.num_of_true_lbls = num_of_true_lbls
        self.pi = None
        self.data = None
        self.centroids = None
        self.true_lbls = None

    def distance(self,x,y):
        if(self.dist_type == 'Euclidean'):
            return np.linalg.norm(x-y)
        elif(self.dist_type == 'Cosine Similarity'):
            return np.dot(x,y) / (np.linalg.norm(x)*np.linalg.norm(y))

    def fit(self,data,true_lbls):
        self.data = data
        self.true_lbls = true_lbls
        # initializing centroids
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        self.centroids = data[indices]

    def computePi(self):
        # for all data points find the closest centroid and update pi
        # reinitializing pi everytime it gets recomputed
        self.pi = np.zeros((len(self.data),self.k), dtype=int)

        for i in range(len(self.data)):
            dist = self.distance(self.data[i],self.centroids[0])
            closest_centroid_idx = 0
            for centroid_idx in range(1,len(self.centroids)):
                if(self.distance(self.data[i],self.centroids[centroid_idx]) < dist):
                    dist = self.distance(self.data[i],self.centroids[centroid_idx])
                    closest_centroid_idx = centroid_idx

            self.pi[i][closest_centroid_idx] = 1

    def computeCentroids(self):
        # for all k clusters
        # pi[i] (reshaped to 1xN) is multiplied with Xi (NxD)
        # normalized by num of data points in cluster k i.e. sum(pi[i])
        for k in range(self.k):
            self.centroids[k] = self.pi.T[k] @ self.data / sum(self.pi.T[k])

    def predict(self):
        # returns cluster lbl allocated to each data point
        iters = 0
        self.computePi()
        self.computeCentroids()
        old_objective_value = float('inf')
        new_objective_value = self.kmeansObjective()

        while iters < self.iters and abs(old_objective_value - new_objective_value) > 1e-6:
            self.computePi()
            if iters != self.iters - 1:
                self.computeCentroids()

            old_objective_value = new_objective_value
            new_objective_value = self.kmeansObjective()
            iters += 1

        print(f'Objective function value: {new_objective_value}')
        return np.argmax(self.pi, axis=1)


    def kmeansObjective(self):
        distances_squared = np.sum((self.data[:, np.newaxis] - self.centroids) ** 2, axis=2) # NxK matrix: Dist of each pt with each centroid
        filtered_distances = distances_squared * self.pi # Element wise multiplication of distances_sq with membership matrix
        return np.sum(filtered_distances) # Sum of all filtered distances

    def evaluteClustering(self):
        # Need to make confusion matrix of algorithm determined cluster indices (row) vs true cluster indices (column)
        # purity = sum of row wise max / total data points
        # Gini index for a row (algorithm determined cluster) [Gj] = 1- sum of(mij/Mj)^2 [i from 1 to number of true cluster]
        # Gini average = Gj * Mj/ total data points

        # creating confusion matrix
        algo_det_lbls = self.predict() # array of shape 1xN
        cm = np.zeros((self.k,self.num_of_true_lbls), dtype=int)

        for i in range(len(algo_det_lbls)):
            algo_det_lbl = algo_det_lbls[i]
            true_lbl = self.true_lbls[i]
            cm[algo_det_lbl][true_lbl] += 1

        Pj_sum = np.sum(np.max(cm,axis=1))
        print(f"Purity: {Pj_sum/len(algo_det_lbls)}")

        Gj = []
        Mj = np.sum(cm,axis=1) # number of data points per cluster
        for i in range(len(cm)):
            mij = np.sum(cm[i] ** 2)
            if(Mj[i] == 0):
                Gj.append(0)
            else:
                Gj.append(1-(mij/Mj[i]**2))

        gini_avg = np.sum(Gj*Mj)/len(algo_det_lbls)
        print(f"Gini Average: {gini_avg}")

In [ ]:
# main.py
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt
from KMeans import KMeans
import pickle
from sklearn.datasets import fetch_20newsgroups

def transform_mnist(data):
       transformed_data = data.reshape(data.shape[0],-1)
       transformed_data[transformed_data>0] = 1
       return transformed_data

def transform_fashion(data):
       transformed_data = data.reshape(data.shape[0],-1)
       return transformed_data

def transform_news_groups(data):
        pass

def visualizeImg(img,lbl,reshaped=False):
        i = 3
        if(reshaped):
               plt.imshow(img.reshape((28,28)), cmap='viridis', interpolation='nearest')
        else:
               plt.imshow(img, cmap='viridis', interpolation='nearest')
        plt.colorbar()  # Show color scale
        plt.title(f'Image of {lbl}')
        plt.xlabel('X-axis')
        plt.ylabel('Y-axis')
        plt.show()

if __name__ == '__main__':

       #imgs = idx2numpy.convert_from_file("Datasets/Fashion /t10k-images-idx3-ubyte")
       #imgs_copy = np.copy(imgs)
       #lbls = idx2numpy.convert_from_file("Datasets/Fashion /t10k-labels-idx1-ubyte")
       #lbls_copy = np.copy(lbls)
       filename = 'Datasets/20 NG/dataset.pkl'

       with open(filename, 'rb') as file:
             ng_data = pickle.load(file)

       newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))
       lbls = newsgroups_test.target
       target_names = newsgroups_test.target_names

       text = np.copy(ng_data)
       labels = np.copy(lbls)

       kmeans = KMeans(k=40,dist_type='Euclidean',iters=10,num_of_true_lbls=20)
       kmeans.fit(data=text,true_lbls=labels)
       kmeans.evaluteClustering()

    #transformed_imgs = transform_mnist(imgs_copy)
    #transformed_imgs = transform_fashion(imgs_copy)
#     i=121
#     visualizeImg(img=imgs_copy[i],lbl=lbls_copy[i])
#     visualizeImg(img=transformed_imgs[i],lbl=lbls[i],reshaped=True)

    #kmeans = KMeans(k=20,dist_type='Euclidean',iters=25,num_of_true_lbls=10)
    #kmeans.fit(data=transformed_imgs,true_lbls=lbls_copy)
    #kmeans.evaluteClustering()

    # MNIST k=10
    #Objective function value: 1485053.0, Purity: 0.2048, Gini Average: 0.8636386695690841

    # MNIST k=5
    #Objective function value: 1500369.0, Purity: 0.189, Gini Average: 0.8555777734123303

    # MNIST k=20
    #Objective function value: 1471162.0, Purity: 0.1993, Gini Average: 0.8580059860829399

    # FASHION k=10
    #Objective function value: 489841403.0, Purity: 0.1618, Gini Average: 0.8811556562125022

    # FASHION k=5
    #Objective function value: 770137032.0, Purity: 0.2008, Gini Average: 0.8542479519654073

    # FASHION k=20
    #Objective function value: 477813985.0, Purity: 0.1627, Gini Average: 0.8795803409135767

    # 20NG k=20
    # Objective function value: 222293.24200732622, Purity: 0.08470525756771109, Gini Average: 0.9328196371857366

    # 20NG k=10
    # Objective function value: 248522.20982834254, Purity: 0.07514604354753053, Gini Average: 0.9416480514077292

    # 20NG k=40
    # RuntimeWarning: invalid value encountered in divide
    # self.centroids[k] = self.pi.T[k] @ self.data / sum(self.pi.T[k])
    # Objective function value: nan, Purity: 0.09240573552841211, Gini Average: 0.930551486020236
    # probably one of the cluster didn't get any points near it. Hence the corresponding col in membership mat is zero

#     centroids = kmeans.centroids
#     for i in range(len(centroids)):
#            visualizeImg(centroids[i],0,reshaped=True)



#How I preprocessed 20 NG

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
import math
from google.colab import drive
drive.mount('/content/drive')
from sklearn.datasets import load_files
import pickle
import numpy as np
import matplotlib.pyplot as plt
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
url = '/content/drive/MyDrive/USML HW2/newsgrps_test.pkl'
url2 = '/content/drive/MyDrive/USML HW2/newsgrps_test_no_footers_headers_quotes.pkl'

In [ ]:
with open(url2, 'rb') as file:
  ng_test = pickle.load(file)

In [ ]:
len(ng_test.data)

7532

In [ ]:
len(ng_test.target)

7532

In [ ]:
i = 10
print(ng_test.data[i],ng_test.target_names[ng_test.target[i]])

From: Greg.Reinacker@FtCollins.NCR.COM
Subject: Windows On-Line Review uploaded
Reply-To: Greg.Reinacker@FtCollinsCO.NCR.COM
Organization: NCR Microelectronics, Ft. Collins, CO
Lines: 12

I have uploaded the Windows On-Line Review shareware edition to
ftp.cica.indiana.edu as /pub/pc/win3/uploads/wolrs7.zip.

It is an on-line magazine which contains reviews of some shareware
products...I grabbed it from the Windows On-Line BBS.

--
--------------------------------------------------------------------------
Greg Reinacker                          (303) 223-5100 x9289
NCR Microelectronic Products Division   VoicePlus 464-9289
2001 Danfield Court                     Greg.Reinacker@FtCollinsCO.NCR.COM
Fort Collins, CO  80525
 comp.os.ms-windows.misc


In [ ]:
i = 0
print(ng_test.data[i])
print(ng_test.target_names[ng_test.target[i]])

I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.
rec.autos


In [ ]:
total_text = ''
for i in range(len(ng_test.data)):
  total_text += ng_test.data[i]

In [ ]:
text = ng_test.data[0]

In [ ]:
len(total_text)

8261569

In [ ]:
from nltk.corpus import wordnet

def is_valid_word(word):
  return bool(wordnet.synsets(word))

total_text = re.sub(r'[^a-zA-Z0-9 \n]', '', total_text)
total_text = re.sub(r'\n+', ' ', total_text)
total_text = total_text.lower()
tokens = word_tokenize(total_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
filtered_tokens = [word for word in tokens if is_valid_word(word)]

In [ ]:
len(filtered_tokens)

601285

In [ ]:
filtered_tokens = set(filtered_tokens)

In [ ]:
len(filtered_tokens)

28892

In [ ]:
filtered_tokens_5k = random.sample(filtered_tokens, 5000)

<ipython-input-70-ffbac7204284>:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  filtered_tokens_5k = random.sample(filtered_tokens, 5000)


In [ ]:
filtered_tokens_5k.sort()

In [ ]:
def preprocess_text(text):
  text = re.sub(r'[^a-zA-Z0-9 \n]', '', text)
  text = re.sub(r'\n+', ' ', text)
  text = text.lower()
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  dict_tokens = dict(Counter(tokens))

  np_arr = np.zeros(5000)
  for i in range(len(filtered_tokens_5k)):
    if filtered_tokens_5k[i] in dict_tokens:
      np_arr[i] = dict_tokens[filtered_tokens_5k[i]]
  return np_arr

In [ ]:
np_dataset = []

for i in range(len(ng_test.data)):
  np_dataset.append(preprocess_text(ng_test.data[i]))

In [ ]:
idx = 0
for i in range(len(np_dataset[idx])):
  if np_dataset[idx][i] != 0:
    print(filtered_tokens_5k[i],np_dataset[idx][i])

book 2.0
heard 2.0
le 1.0
much 2.0
usually 1.0
words 1.0
year 1.0


In [ ]:
dataset = np.array(np_dataset)

In [ ]:
dataset.shape

(7532, 5000)

In [ ]:
dataset.nbytes

301280000

In [ ]:
zero_count = np.count_nonzero(dataset == 0)

# Calculate the total number of elements in the matrix
total_elements = dataset.size

# Calculate the sparsity (percentage of zero elements)
sparsity = (zero_count / total_elements) * 100

print(f"The sparsity of the dataset is {sparsity:.2f}%")

The sparsity of the dataset is 99.82%


In [ ]:
filename = '/content/drive/MyDrive/USML HW2/dataset.pkl'

with open(filename, 'wb') as file:
  pickle.dump(dataset, file)


In [ ]:
idx = 0
for i in range(len(dataset[idx])):
  if dataset[idx][i] != 0:
    print(filtered_tokens_5k[i],dataset[idx][i])

book 2.0
heard 2.0
le 1.0
much 2.0
usually 1.0
words 1.0
year 1.0


In [ ]:
print(ng_test.data[0])
print(ng_test.target_names[ng_test.target[0]])

I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.
rec.autos
